In [9]:
import pandas as pd
from tabulate import tabulate
class Trader:
    def __init__(self,initial_balance=100000):
        self.balance = initial_balance
        self.RiskToReward  = "1:2"
        self.trad_book = pd.DataFrame(columns=['IndexName', 'TradSide', 'Status', 'Quantity', 'BuyPrice',
                                               'BuyDatetime', 'SellPrice', 'SellDatetime','SLValue',
                                               'TargetValue', 'PnL Status','PnL'])
        

    def backtest(self, df):
        open_order = None
        TradSide = None
        tg_order = None
        sl_order = None

        for index, row in df.iterrows():
            if row["TradSide"] != "None" and open_order is None:
                open_order = row["Close"]
                TradSide = row["TradSide"]
                sl = 50 * int(self.RiskToReward.split(":")[0])
                tg = sl * int(self.RiskToReward.split(":")[1])
                sl_order, tg_order = (open_order - sl, open_order + tg) if TradSide == "CE" else (open_order + sl, open_order - tg)
                symbol = "Nifty50"
                quantity = 10
                new_log = pd.DataFrame({'IndexName': [symbol],'TradSide':[TradSide],'Status':["Open"] ,'Quantity': [quantity], 'BuyPrice': [open_order],
                                'BuyDatetime': [row["Datetime"]], 'SellDatetime': [None], 'SellPrice': [None],
                                'PnL': [0],'SLValue': [sl_order], 'TargetValue': [tg_order], 'PnL Status': [None] })
                self.trad_book = pd.concat([self.trad_book, new_log], ignore_index=True)
            if open_order is not None:
                if (TradSide == "CE" and (row["Close"] >= tg_order or row["Close"] <= sl_order)) or \
                   (TradSide == "PE" and (row["Close"] <= tg_order or row["Close"] >= sl_order)):
                    pnl = row["Close"] - open_order if TradSide == "CE" else open_order - row["Close"]
                    self.balance += pnl
                    
                    index = self.trad_book.shape[0] -1
                    self.trad_book.at[index, 'SellPrice'] = row["Close"]
                    self.trad_book.at[index, 'SellDatetime'] = row["Datetime"]
                    self.trad_book.at[index, 'Status'] = "Done"
                    self.trad_book.at[index, 'PnL'] = pnl
                    self.trad_book.at[index, 'PnL Status'] = "Profit" if  pnl >0  else "Loss"
                    self.trad_book.at[index, 'SellDatetime'] = row["Datetime"]
                    open_order = None

    
    def stats(self):
            df = self.trad_book
            total_trade = len(df.index)
            pnl = df.PnL.sum()
            winners = len(df[df.PnL > 0])
            losers = len(df[df.PnL <= 0])
            win_ratio = round((winners / total_trade) * 100, 2)
    
            # Calculate CE and PE trades
        
            ce_trades = f"{(len(df[(df['TradSide'] == 'CE') & (df['PnL'] > 0)]) / len(df[df['TradSide'] == 'CE']) * 100):.2f}%" if len(df[df['TradSide'] == 'CE']) != 0 else 0
            pe_trades = f"{(len(df[(df['TradSide'] == 'PE') & (df['PnL'] > 0)]) / len(df[df['TradSide'] == 'PE']) * 100):.2f}%" if len(df[df['TradSide'] == 'PE']) != 0 else 0

    
            # Calculate additional metrics
            capital = self.balance
            max_win = round(df[df.PnL > 0].PnL.max(), 2) if winners > 0 else 0
            max_profit_sum = round(df[df.PnL > 0].PnL.sum(), 2) if winners > 0 else 0
            max_loss = round(df[df.PnL <= 0].PnL.min(), 2) if losers > 0 else 0
            max_loss_sum = round(df[df.PnL <= 0].PnL.sum(), 2) if losers > 0 else 0
            total_profit = round(df.PnL.sum(), 2)
            total_profit_percentage = round((total_profit / self.balance) * 100, 2)
            trading_days = pd.to_datetime(df['BuyDatetime']).dt.strftime('%Y%m%d').nunique()

    
            # Prepare the data for tabular representation
            # parameters = ['Total Trades', 'Capital', 'Total Wins', 'Total Losses', 'Win Ratio',
            #               'Max Win', 'Max Win Score','Max Loss', 'Max Loss Score','Total Profit', ' Grow Profit %',
            #               'CE Trades', 'PE Trades', 'Stategy Name']


            parameters = ['Total Trading Days','Total Trades','RiskToReward','Capital', 'Total Wins', 'Total Losses', 'Win Ratio','Total Profit', 'Grow Profit %',
                              'Max Win', 'Max Win Score','Max Loss', 'Max Loss Score','CE Trades', 'PE Trades', 'Stategy Name']


        
            data_points = [trading_days,total_trade, self.RiskToReward ,capital, winners, losers, f"{win_ratio}%",total_profit,f"{total_profit_percentage}%",
                           max_win, max_profit_sum,max_loss, max_loss_sum,ce_trades, pe_trades, "Test"]

        
            data = list(zip(parameters, data_points))
    
            # Print the tabular representation
            print(tabulate(data, headers=['Parameters', 'Values'], tablefmt='psql'))
    
            excel_file = "statistics.csv"
            try:
                existing_df = pd.read_csv(excel_file)
                new_data = pd.DataFrame([data_points], columns=parameters)
                updated_df = pd.concat([existing_df, new_data], ignore_index=True)
                updated_df.to_csv(excel_file, index=False)
            except FileNotFoundError:
                new_data = pd.DataFrame([data_points], columns=parameters)
                new_data.to_csv(excel_file, index=False)

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("MainDataSet15m.csv")

In [18]:
df['Prev_Candle_Signal'] =df['Candle_Signal'].shift(1)
df['Prev_Candle'] = df['Candle'].shift(1)
df = df.reindex(sorted(df.columns), axis=1)

In [29]:
ce_condition = ((df['High'].shift(1) < df['15EMA'].shift(1)) & (df["Close"] > df['15EMA'])
                # & (df["RSI"] <= 65)
                # &(df["SuperTrend"] == 1)
                &(df["Prev_Candle_Signal"] == "Bullish")
               )

pe_condition = ((df['Low'].shift(1) > df['15EMA'].shift(1)) & (df["Close"] < df['15EMA'])
                # &(df["RSI"] >= 35)
                # &(df["SuperTrend"] == -1)
                &(df["Prev_Candle_Signal"] == "Bearish")
               )

df['TradSide'] = np.select([ce_condition, pe_condition], ['CE', 'PE'], default='None')

In [30]:
# ce_condition = (df['First_Candle_Low'] > df['Close'] ) & ()
# pe_condition = (df['First_Candle_High'] < df['Close'])

# df['TradSide'] = np.select([ce_condition, pe_condition], ['CE', 'PE'], default='None')

In [ ]:
trader = Trader()
trader.backtest(df)
trader.stats()

In [ ]:
# +--------------------+--------------------+
# | Parameters         | Values             |
# |--------------------+--------------------|
# | Total Trading Days | 449                |
# | Total Trades       | 696                |
# | RiskToReward       | 1:2                |
# | Capital            | 104027.69000000009 |
# | Total Wins         | 301                |
# | Total Losses       | 395                |
# | Win Ratio          | 43.25%             |
# | Total Profit       | 4027.69            |
# | Grow Profit %      | 3.87%              |
# | Max Win            | 733.6              |
# | Max Win Score      | 50431.58           |
# | Max Loss           | -734.2             |
# | Max Loss Score     | -46403.89          |
# | CE Trades          | 44.73%             |
# | PE Trades          | 41.74%             |
# | Stategy Name       | Test               |
# +--------------------+--------------------+

In [22]:
tk = trader.trad_book
tk

,IndexName,TradSide,Status,Quantity,BuyPrice,BuyDatetime,SellPrice,SellDatetime,SLValue,TargetValue,PnL Status,PnL
0,Nifty50,PE,Done,10,42933.40,2021-11-24 14:00:00,42783.1,2021-11-24 14:30:00,42983.40,42833.40,Profit,150.3
1,Nifty50,CE,Done,10,42726.50,2021-11-25 11:15:00,42630.6,2021-11-25 11:30:00,42676.50,42826.50,Loss,-95.9
2,Nifty50,PE,Done,10,42619.20,2021-11-25 12:15:00,42489.0,2021-11-25 12:45:00,42669.20,42519.20,Profit,130.2
3,Nifty50,PE,Done,10,42626.10,2021-11-25 15:00:00,41892.5,2021-11-26 09:15:00,42676.10,42526.10,Profit,733.6
4,Nifty50,CE,Done,10,41141.40,2021-11-29 11:00:00,41276.3,2021-11-29 11:30:00,41091.40,41241.40,Profit,134.9
...,...,...,...,...,...,...,...,...,...,...,...,...
691,Nifty50,CE,Done,10,53936.02,2024-05-13 11:30:00,54104.36,2024-05-13 12:15:00,53886.02,54036.02,Profit,168.34
692,Nifty50,PE,Done,10,53930.11,2024-05-13 13:15:00,54461.99,2024-05-13 13:45:00,53980.11,53830.11,Loss,-531.88
693,Nifty50,PE,Done,10,53933.10,2024-05-13 09:15:00,53800.12,2024-05-13 09:30:00,53983.10,53833.10,Profit,132.98
694,Nifty50,CE,Done,10,53936.02,2024-05-13 11:30:00,54104.36,2024-05-13 12:15:00,53886.02,54036.02,Profit,168.34


In [16]:
ViewList = ["Datetime", "Day_Prev_Candle_Signal", "Day_Prev_Candle", "Day_RSI", "RSI", "Prev_Candle_Signal", "Prev_Candle", "TradSide"]
row_data = df[df["Datetime"].isin(tk["BuyDatetime"])].merge(tk[["BuyDatetime", "PnL Status"]], left_on="Datetime", right_on="BuyDatetime", how="inner")[ViewList + ["PnL Status"]]

In [193]:
# an_side = "CE"
# Column_name = "RSI"

# import matplotlib.pyplot as plt

# # Filter data for the specified trading side and PnL Status
# df_loss = row_data[(row_data["TradSide"] == an_side) & (row_data["PnL Status"] == "Loss")]
# df_profit = row_data[(row_data["TradSide"] == an_side) & (row_data["PnL Status"] == "Profit")]

# # Create a figure and axis for subplots
# fig, axs = plt.subplots(1, 2, figsize=(6, 3))

# # Plot histograms for the Loss and Profit categories side by side
# axs[0].hist(df_loss[Column_name], bins=20, color='red', alpha=0.7)
# axs[0].set_title('Loss')
# axs[0].set_xlabel(f"{Column_name} {an_side}")
# axs[0].set_ylabel('Frequency')

# axs[1].hist(df_profit[Column_name], bins=20, color='green', alpha=0.7)
# axs[1].set_title('Profit')
# axs[1].set_xlabel(f"{Column_name} {an_side}")
# axs[1].set_ylabel('Frequency')

# plt.tight_layout()
# plt.show()


In [194]:
# time_enter = "2024-05-13 09:15"
# df[df["Datetime"].str.contains(time_enter)].iloc[-1]